# Data

In [16]:
# from datasets import load_dataset
# 
# data = load_dataset("takiholadi/kill-me-please-dataset")
# data

DatasetDict({
    train: Dataset({
        features: ['text', 'tags', 'votes', 'url', 'datetime'],
        num_rows: 27321
    })
    test: Dataset({
        features: ['text', 'tags', 'votes', 'url', 'datetime'],
        num_rows: 2772
    })
})

In [61]:
# train_texts = data["train"]["text"]
# test_texts = data["test"]["text"]
# 
# characters = set()
# for text in train_texts:
#     characters.update(text)
# 
# for text in test_texts:
#     characters.update(text)
# characters = sorted(list(characters))
# vocab_size = len(characters)
# 
# ''.join(characters)

'\t\n !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|~\xa0¥©«°±»¾×àë́ЁАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё\u200b\u200c–—―‘’“”„…€₽№™−！（）－'

Не будем пока использовать этот dataset

In [80]:
!wsl wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-01-10 15:18:10--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

     0K .......... .......... .......... .......... ..........  4%  868K 1s
    50K .......... .......... .......... .......... ..........  9% 1.31M 1s
   100K .......... .......... .......... .......... .......... 13% 3.08M 1s
   150K .......... .......... .......... .......... .......... 18% 1.88M 1s
   200K .......... .......... .......... .......... .......... 22% 12.8M 0s
   250K .......... .......... .......... .......... .......... 27% 1.78M 0s
   300K .......... .......... .......... .......... .......... 32% 18.1M 0s
   350K .......... ..

In [83]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters:", len(text))

length of dataset in characters: 1115394


In [85]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [86]:
characters = sorted(list(set(text)))
vocab_size = len(characters)

''.join(characters)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [88]:
s2i = {char: i for i, char in enumerate(characters)}
i2s = {i: char for i, char in enumerate(characters)}

encode = lambda text: [s2i[char] for char in text]
decode = lambda text: ''.join([i2s[char] for char in text])

encode('hi, nigger'), decode(encode('hi, nigger'))

([46, 47, 6, 1, 52, 47, 45, 45, 43, 56], 'hi, nigger')

In [90]:
import tiktoken

# Не будем использовать, но вообще это SOTA

enc = tiktoken.encoding_for_model("gpt-4o")
enc.encode('hi, nigger'), enc.decode(enc.encode('hi, nigger'))

([3686, 11, 297, 6895], 'hi, nigger')

In [92]:
import torch

tensor_data = torch.tensor(encode(text))
train_tensor = tensor_data[:int(0.9 * len(tensor_data))]
test_tensor = tensor_data[int(0.9 * len(tensor_data)):]
train_tensor.shape, test_tensor.shape

(torch.Size([1003854]), torch.Size([111540]))

In [96]:
block_size = 8
x = train_tensor[:block_size]
y = train_tensor[1:block_size + 1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"When input is {context} the Target: {target}")

When input is tensor([18]) the Target: 47
When input is tensor([18, 47]) the Target: 56
When input is tensor([18, 47, 56]) the Target: 57
When input is tensor([18, 47, 56, 57]) the Target: 58
When input is tensor([18, 47, 56, 57, 58]) the Target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the Target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the Target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the Target: 58


In [109]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8


def get_batch(split):
    data = train_tensor if split == 'train' else test_tensor
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print(' ---- ')

for b in range(batch_size):  # batch dimension
    for t in range(block_size):  # time dimension
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} the Target: {target}")
    print()

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
 ---- 
When input is [24] the Target: 43
When input is [24, 43] the Target: 58
When input is [24, 43, 58] the Target: 5
When input is [24, 43, 58, 5] the Target: 57
When input is [24, 43, 58, 5, 57] the Target: 1
When input is [24, 43, 58, 5, 57, 1] the Target: 46
When input is [24, 43, 58, 5, 57, 1, 46] the Target: 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] the Target: 39

When input is [44] the Target: 53
When input is [44, 53] the Target: 56
When input is [44, 53, 56] the Target: 1
When input is [44, 53, 56, 1] the Target: 58
When input is [44, 53, 56, 1, 58] the Target: 46
When input is [44,

In [110]:
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [124]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx Is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=- 1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [130]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [150]:
eval_iters = 100


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [155]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data

    # every once in a while evaluate the loss on train and val sets
    if steps % 1000 == 0:
        losses = estimate_loss()
        print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 2.4603, val loss 2.5110
step 1000: train loss 2.4531, val loss 2.4810
step 2000: train loss 2.4530, val loss 2.4799
step 3000: train loss 2.4542, val loss 2.4980
step 4000: train loss 2.4666, val loss 2.4899
step 5000: train loss 2.4596, val loss 2.4864
step 6000: train loss 2.4506, val loss 2.4925
step 7000: train loss 2.4436, val loss 2.4919
step 8000: train loss 2.4508, val loss 2.5037
step 9000: train loss 2.4491, val loss 2.5012


In [205]:
idx = torch.tensor([[5]], dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=5)[0].tolist()))

'ser, 


## Математический трюк self-attention

In [207]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.rand(B, T, C)
x.shape

torch.Size([4, 8, 2])